# Подключение библиотек

In [ ]:
import os
import pandas as pd
import csv
import numpy as np
from datetime import datetime
import datetime
import matplotlib
import matplotlib.pyplot as plt
import requests
import urllib.parse

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
directory = '/content/drive/My Drive/'

Mounted at /content/drive


In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2022-04-05 10:21:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  49.9MB/s    in 0.3s    

2022-04-05 10:21:55 (49.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!/ngrok authtoken 27KPm57VtpkXsmXrUisbJvSsbug_2BRGqRFzgpAT2KqNa2HHK

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install Flask==0.12.2 --upgrade
!pip install Flask-SQLAlchemy==2.2 --upgrade

     |████████████████████████████████| 83 kB 1.1 MB/s 
  Attempting uninstall: Flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [ ]:
from flask import Flask, render_template, jsonify
from flask import request
from flask_ngrok import run_with_ngrok
import socket
from flask import Response

import sqlalchemy
from sqlalchemy import MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.orm import sessionmaker,Session
from flask_sqlalchemy import SQLAlchemy
import json
import datetime as dt

In [ ]:
measurements_path = directory + "hawaii_measurements.csv"
stations_path = directory +  "hawaii_stations.csv"

# Создание экземпляров SQLAlchemy

In [ ]:
engine = create_engine('sqlite:///orm.sqlite')#, echo =True)
Base = declarative_base()

In [ ]:
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    date = Column(String(15))
    prcp = Column(Float)
    tobs = Column(Integer)

class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    name = Column(String(45))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [ ]:
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [ ]:
measurements_df = pd.read_csv(measurements_path)
stations_df = pd.read_csv(stations_path)

In [ ]:
measurements = []
for index, row in measurements_df.iterrows():
  measurement = Measurements(station=row['station'], date=row['date'], prcp=row['prcp'], tobs=row['tobs'])
  measurements.append(measurement)

session.bulk_save_objects(measurements)
session.commit()

In [ ]:
stations = []
for index, row in stations_df.iterrows():
  station = Station(station=row['station'], name=row['name'], latitude=row['latitude'],longitude=row['longitude'],elevation=row['elevation'])
  stations.append(station)

session.bulk_save_objects(stations)
session.commit()

# Миграция в Flask-project

In [ ]:
engine = create_engine('sqlite:///orm.sqlite')
Base = automap_base()
Base.prepare(engine, reflect=True)

Measurements = Base.classes.measurements
Station = Base.classes.station
session = Session(engine)

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def index():
  return (f"API для отображения данных на конкретную дату (в формате Y-m-d):<br/>"
      f"/api/v1.0/<start><br/><br/>"
      f"API для отображения данных между двумя датами (в формате Y-m-d):<br/>"
      f"/api/v1.0/<start>/<end><br/><br/>"
      f"Доступные API с данными:<br/>"
      f"/api/v1.0/precipitation<br/>"
      f"/api/v1.0/stations<br/>"
      f"/api/v1.0/tobs<br/>")

@app.route('/api/v1.0/precipitation')
def precipitation():
  last_date = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
  last_date=datetime.datetime.strptime(last_date[0],"%Y-%m-%d")
  last_year = datetime.date(2017, 8, 23) - datetime.timedelta(days=365)
  rain = session.query(Measurements.date, Measurements.prcp).filter((Measurements.date >= last_year)&(Measurements.date<=last_date)).order_by(Measurements.date).all()
  rain_totals = []
  for x_rain in rain:
    row = {}
    row['date'] = x_rain[0]
    row['prcp'] = x_rain[1]
    rain_totals.append(row)
  return json.dumps(rain_totals)

@app.route("/api/v1.0/stations")
def get_stations():
  station_query = session.query(Station.name, Station.station).all()
  station_list =[]
  sd = {}
  for row in station_query:
    sd['station code']=row[0]
    sd['station name']=row[1]
    station_list.append(sd)
  return json.dumps(station_list,separators=(',',':'))

@app.route("/api/v1.0/tobs")
def get_prev_temp():
  last_date = session.query(Measurements.date, Measurements.tobs)
  last_year = datetime.date(2017, 8, 23) - datetime.timedelta(days=365)
  prev_temp_query = session.query(Measurements.tobs, Measurements.date, Measurements.station).filter(Measurements.date > last_year).all()
  prev_list =[]
  pd = {}
  for row in prev_temp_query:
    pd['station code']=row[2]
    pd['date']=row[1]
    pd['temp']=row[0]
    prev_list.append(pd)
  return json.dumps(prev_list, separators=(',',':'))

@app.route("/api/v1.0/<start>")
def start(start):
  start_date=dt.datetime.strptime(start, '%Y-%m-%d').date()
  ave_tobs=session.query(func.avg(Measurements.tobs)).filter(Measurements.date>=start_date).all()[0][0]
  min_tobs=session.query(func.min(Measurements.tobs)).filter(Measurements.date>=start_date).all()[0][0]
  max_tobs=session.query(func.max(Measurements.tobs)).filter(Measurements.date>=start_date).all()[0][0]
  return "Средняя температура, начинающаяся с {}, равна {}.\nМинимальная температура, наблюдаемая из {}, равна {}.\n Максимальный темп,начинающийся с {}, равен {}.".format(start,ave_tobs,start,min_tobs,start,max_tobs)

@app.route("/api/v1.0/<start>/<end>")
def between(start,end):
  start_date=dt.datetime.strptime(start, '%Y-%m-%d').date()
  end_date=dt.datetime.strptime(end, '%Y-%m-%d').date()
  ave_tobs=session.query(func.avg(Measurements.tobs)).filter((Measurements.date>=start_date)&(Measurements.date<=end_date)).all()[0][0]
  min_tobs=session.query(func.min(Measurements.tobs)).filter((Measurements.date>=start_date)&(Measurements.date<=end_date)).all()[0][0]
  max_tobs=session.query(func.max(Measurements.tobs)).filter((Measurements.date>=start_date)&(Measurements.date<=end_date)).all()[0][0]
  return "Средняя температура между {} и {} равна {}.Минимальная температура между {} и {} равна {}.Максимальная температура между {} и {} равна {}".format(start,end,ave_tobs,start,end,min_tobs,start,end,max_tobs)

if __name__ == '__main__':
  app.run()